In [ ]:
import modi
import time


ModuleNotFoundError: ignored

In [ ]:
bundle = modi.MODI()
bundle.modules

NameError: ignored

In [ ]:
# modules

# gyro
gyro = bundle.gyros[0]


# button
button = bundle.buttons[0]

# mic
mic = bundle.mics[0]

# infrared sensor
ir = bundle.irs[0]

# ultrasonic sensor
ultrasonic = bundle.ultrasonics[0]

# dial
dial = bundle.dials[0]

# speaker
speaker = bundle.speakers[0]

# display
display = bundle.displays[0]

In [ ]:
import subprocess  # 電話機能のapiのため、import

global emergency

def ang_vel(a, b):
    global ang_vel_check
    if abs(gyro.angular_vel_x) <= a and abs(gyro.angular_vel_y) <= a and abs(gyro.angular_vel_z) <= a:
        ang_vel_check = True

    elif abs(gyro.angular_vel_x) >= b or abs(gyro.angular_vel_y) >= b or abs(gyro.angular_vel_z) >= b:
        ang_vel_check = False
        emergency = True

        if emergency:
            subprocess.run('call.py', shell = True)

        while emergency:  # 危険な状態と認識した時スピーカーから1秒感覚で持続的な音を出力
            speaker.tune = 3000, 20

            if button.clicked == 1:  # ボタンをもう一度押したら強制終了、スピーカーオフ
                speaker.turn_off()
                break
    else:
        ang_vel_check = False

    print("angular_velocity: ", ang_vel_check)
    return ang_vel_check

def tilt(a):
    global tilt_check
    if abs(gyro.pitch) <= a and abs(gyro.roll) <= a:  # バーベルの傾きの程度を把握するためにジャイロスコープのroll, pitch, yawを使用
        tilt_check = True

    elif a < abs(gyro.pitch) and a < abs(gyro.roll):
        tilt_check = False
        emergency = True

        if emergency:
            subprocess.run('call.py', shell = True)

        while emergency:  # 危険な状態と認識した時スピーカーから1秒感覚で持続的な音を出力
            speaker.tune = 3000, 20

            if button.clicked == 1:  # ボタンをもう一度押したら強制終了、スピーカーオフ
                speaker.turn_off()
                break
    else:
        tilt_check = False

    print("tilt: ", tilt_check)
    return tilt_check

def vibration(a, b):
    global vibration_check
    if 0 <= abs(gyro.vibration) <= a:  # 振動数を計測し、10を超えた時バーベルの震えが激しいため姿勢が不安定だと判断
        vibration_check = True

    elif a < gyro.vibration <= b:
        vibration_check = False

    else:  # 振動数が閾値を超えた時、危険と判断->emergency
        vibration_check = False
        emergency = True

        if emergency:
            subprocess.run('call.py', shell = True)

        while emergency:  # 危険な状態と認識した時スピーカーから1秒感覚で持続的な音を出力
            speaker.tune = 3000, 20

            if button.clicked == 1:  # ボタンをもう一度押したら強制終了、スピーカーオフ
                speaker.turn_off()
                break

    print("vibration: ", vibration_check)
    return vibration_check

def distance(chest_distance):  # ベンチプレスモード用
    global distance_check
    if chest_distance*0.5 <= ir.proximity <= chest_distance*1.5:
        distance_check = True

    else:  # 距離測定シークエンスでは閾値設定の必要なし
        distance_check = False

    print("distance: ", distance_check)
    return distance_check

def ultrasonic_distance(face_distance):  # デッドリフトモード用
    global us_distance_check
    if face_distance*0.5 <= ir.proximity <= face_distance*1.5:
        us_distance_check = True

    else:  # 距離測定シークエンスでは閾値の設定必要なし
        us_distance_check = False

    print("ultrasonic_distance: ", us_distance_check)
    return us_distance_check



def Bench_Press():

    chest_distance = 8  # バーベルを下ろした時、赤外線センサーと胸部の間の距離の基準となる値
    reps_goal = 2　　　　　#　回数の目標値
    reps_limit = 3  # 12回をしっかりやらせるために20回のチャンスを付与
    set_cnt = 0  # セット数のカウンター
    set_goal = 1  # セット数の目標値
    point_goal = 2  # 観測点の数や運動の種類関係なく、2回通過したら1回とする
    pose_monitor_1 = False  # 観測点1の姿勢モニタリングの結果
    pose_monitor_2 = False  # 観測点2の姿勢モニタリングの結果
    set_monitor = False  # セットごとにモニタリング結果を表示
    set_failcheck = [False]  # 3セットを通してpass/falseの回数を各自カウント
    emergency = False　　　　　#　緊急状態シークエンスに移行するための変数

    print("Bench press mode")
    display.text = "Bench press mode"
    while button.double_clicked !=1:
    # 間違ったモード選択をした際に、ボタンを2回押すとモード選択シークエンスに戻る

        if button.clicked == 1 :

            print("Bench press starts after beep:")
            display.text = "Bench press starts after beep:"
            time.sleep(3)  # 初期姿勢のために10秒の時間を付与
            display.clear()


            speaker.tune = 3000, 50
            time.sleep(0.1)
            speaker.turn_off()
            # ぴーっという音の後で回数カウントがスタート
            time.sleep(0.5)

            while set_cnt < set_goal:  # セット数目標値に到達するまで繰り返す

                reps_cnt = 0  # 1セットごとの回数初期化
                pass_cnt = 0  # reps 총과 횟수 초기화　1セっトごとの回数初期化　　

                while reps_cnt < reps_limit:  # 횟수 목표치를 채울 때까지 반복　回数目標値に到達するまで繰り返す
                    if reps_cnt != 0:
                        time.sleep(1)  # セット間のインターバル50秒
                    i = 0  # 기점 숫자 초기화観測点の数字初期化
                    while i < point_goal:

                        if mic.volume > 50:  # マイクから突然50以上のボリュームを持つ音（器具を落とす音や悲鳴など）が入力されたときに閾値を超えたと判断し危険状況とみなす
                            emergency = True

                            if emergency:
                                subprocess.run('call.py', shell = True)

                            while emergency:
                                speaker.tune = 3000, 20


                                if button.clicked == 1:
                                    speaker.turn_off()
                                    break

                        elif 10 <= mic.volume <= 50:  # バーバルを上げる時、下げる時の呼吸の音で観測点を記録

                            if i == 0:
                                ang_vel(10, 20)
                                tilt(30)
                                vibration(5, 10)
                                distance(chest_distance)

                                if ang_vel_check and tilt_check and vibration_check and distance_check:
                                    pose_monitor_1 = True
                                    print(pose_monitor_1)

                                else:
                                    pose_monitor_1 = False
                                    print(pose_monitor_1)

                            elif i == 1:
                                ang_vel(10, 20)
                                tilt(30)
                                vibration(5, 10)

                                if ang_vel_check and tilt_check and vibration_check:
                                    pose_monitor_2 = True
                                    print(pose_monitor_2)

                                else:
                                    pose_monitor_2 = False
                                    print(pose_monitor_2)

                            i += 1
                            time.sleep(0.5)

                        else:
                            pass
                        time.sleep(0.5)

                    #############################################################　観測値サイクル終了　##############################################################
                    if pose_monitor_1 and pose_monitor_2:  # 2つの観測点を両方通った時に回数をプラス
                        pass_cnt += 1


                        if pass_cnt == reps_goal:  # 回数の目標値に達したとき、reps_limitに関係なく予めループを脱出し、1セット終了
                            display.text = "O"
                            print("o")
                            time.sleep(0.5)
                            display.clear()
                            set_failcheck[set_cnt] = True  # 該当するセットをクリアしたと判断
                            if set_cnt != set_goal:
                                print("Congratulations! Prepare for next set.")
                                display.text = "Congratulations! Prepare for next set."
                                time.sleep(2)
                                display.clear()
                            set_cnt += 1
                            break

                        display.text = "O"
                        print("o")
                        time.sleep(0.5)
                        display.clear()
                        reps_cnt += 1

                    else:  # 2つの観測点のうち、1つでも通過できなかった場合
                        display.text = "X"
                        print("x")
                        time.sleep(0.5)
                        display.clear()
                        reps_cnt += 1
                #############################　reps_limit サイクル終了　#######################################

                if pass_cnt < reps_goal:  # reps_limitほど繰り返したのに通過した回数が目標値を達成できなければ失敗
                    print("Failed ... Try harder next set")
                    display.text = "Failed ... Try harder next set"
                    set_monitor = False
                    time.sleep(2)
                    display.clear()
                    set_cnt += 1



            ############################################　set_goal サイクル終了　######################################

            if set_failcheck == [True]:
                print("You are a bench press master!")
                display.text = "You are a bench press master!"
                time.sleep(2)
                display.clear()
                break

            else:
                print("Try harder!")
                display.text = "Try harder!"
                time.sleep(2)
                display.clear()
                break
#####################################################################　ベンチプレスモード終了　###############################################################

def Squat():
    reps_goal = 2
    reps_limit = 3  # 12回をしっかりやらせるために20回のチャンスを付与
    set_cnt = 0  # セット数カウンター
    set_goal = 1
    point_goal = 2  # 観測点の数や運動の種類関係なく、2回通過したら1回とする
    pose_monitor_1 = False　# 観測点1の姿勢モニタリングの結果
    pose_monitor_2 = False # 観測点2の姿勢モニタリングの結果
    set_failcheck = [False]  # 3セットを通してpass/falseの回数を各自カウント
    emergency = False     #　緊急状態シークエンスに移行するための変数

    print("Squat mode")
    display.text = "Squat mode"
    while button.double_clicked !=1:
    # 間違ったモード選択をした際に、ボタンを2回押すとモード選択シークエンスに戻る

        if button.clicked == 1 :

            print("Squat starts sfter beep:")
            display.text = "Squat starts after beep:"
            time.sleep(3)  # 初期姿勢のために10秒の時間を付与
            display.clear()


            speaker.tune = 3000, 50
            time.sleep(0.1)
            speaker.turn_off()
            # ぴーっという音の後で回数カウントがスタート

            while set_cnt < set_goal:  # セット数目標値に到達するまで繰り返す

                reps_cnt = 0  # 1セットごとの回数初期化
                pass_cnt = 0  # repsの回数初期化

                while reps_cnt < reps_limit:  # セット数目標値に到達するまで繰り返す
                    if reps_cnt != 0:
                        time.sleep(1)  # 1セットが終わったら50秒ずつ休息の時間付与

                    i = 0  # 기점 숫자 초기화

                    while i < point_goal:


                        if mic.volume > 50:  # マイクから突然50以上のボリュームを持つ音（器具を落とす音や悲鳴など）が入力されたときに閾値を超えたと判断し危険状況とみなす
                            emergency = True

                            if emergency:
                                subprocess.run('call.py', shell = True)

                            while emergency:
                                speaker.tune = 3000, 20


                                if button.clicked == 1:
                                    speaker.turn_off()
                                    break

                        elif 10 <= mic.volume <= 50:  # バーバルを上げる時、下げる時の呼吸の音で観測点を記録

                            if i == 0:
                                ang_vel(10, 20)
                                tilt(30)
                                vibration(5, 10)


                                if ang_vel_check and tilt_check and vibration_check:
                                    pose_monitor_1 = True
                                    print(pose_monitor_1)

                                else:
                                    pose_monitor_1 = False
                                    print(pose_monitor_1)

                            elif i == 1:
                                ang_vel(10, 20)
                                tilt(30)
                                vibration(5, 10)

                                if ang_vel_check and tilt_check and vibration_check:
                                    pose_monitor_2 = True
                                    print(pose_monitor_2)

                                else:
                                    pose_monitor_2 = False
                                    print(pose_monitor_2)

                            i += 1
                            time.sleep(0.5)

                        else:
                            pass
                        time.sleep(0.5)

                    #############################################################　観測値サイクル終了　##############################################################
                    if pose_monitor_1 and pose_monitor_2:  # 2つの観測点を両方通った時に回数をプラス
                        pass_cnt += 1

                        if pass_cnt == reps_goal:  # 回数の目標値に達したとき、reps_limitに関係なく予めループを脱出し、1セット終了
                            display.text = "O"
                            print("o")
                            time.sleep(0.5)
                            display.clear()
                            set_failcheck[set_cnt] = True  # 該当するセットをクリアしたと判断
                            if set_cnt != set_goal:
                                print("Congratulatons! Prepare for next set.")
                                display.text = "Congratulations! Prepare for next set."
                                time.sleep(2)
                                display.clear()
                            set_cnt += 1
                            break

                        display.text = "O"
                        print("o")
                        time.sleep(0.5)
                        display.clear()
                        reps_cnt += 1

                    else:  # 2つの観測点のうち一つでも通過できなかった場合
                        display.text = "X"
                        print("x")
                        time.sleep(0.5)
                        display.clear()
                        reps_cnt += 1
                #############################　reps_limit サイクル終了　#######################################

                if pass_cnt < reps_goal:  # reps_limitほど繰り返したのに通過した回数が目標値を達成できなければ失敗
                    print("Failed ... Try harder next set")
                    display.text = "Failed ... Try harder next set"
                    time.sleep(2)
                    display.clear()
                    set_cnt += 1



            ############################################　set_goal サイクル終了######################################



            if set_failcheck == [True]:
                print("You are a squat master!")
                display.text = "You are a squat master!"
                time.sleep(2)
                display.clear()
                break

            else:
                print("Try harder!")
                display.text = "Try harder!"
                time.sleep(2)
                display.clear()
                break
#####################################################################スクワッドモード終了###############################################################

def Deadlift():　　　　　　　　　　　　　　　　　
    set_cnt = 0  # セット数カウント
    set_goal = 1
    point_goal = 2  # 観測点の数や運動の種類関係なく、2回通過したら1回とする
    pose_monitor_1 = False
    pose_monitor_2 = False
    set_failcheck = [False]  # 3セットを通してpass/falseの回数を各自カウント
    emergency = False

    print("Deadlift mode")
    display.text = "Deadlift mode"
    while button.double_clicked !=1:
     # 間違ったモード選択をした際に、ボタンを2回押すとモード選択シークエンスに戻る
        if button.clicked == 1 :

            print("Deadlift starts after beep:")
            display.text = "Deadlift starts after beep:"
            time.sleep(3)  # 初期姿勢のために10秒の時間を付与
            display.clear()


            speaker.tune = 3000, 50
            time.sleep(0.1)
            speaker.turn_off()
            # ぴーっという音の後で回数カウントがスタート

            while set_cnt < set_goal:  # セット数目標値に到達するまで繰り返す

                reps_cnt = 0  # 1セットごとの回数初期化
                pass_cnt = 0  # repsの回数初期化

                while reps_cnt < reps_limit:  # 回数目標値に到達するまで繰り返す
                    if reps_cnt != 0:
                        time.sleep(1)  # 1セットが終わったら50秒の休息を付与

                    i = 0  # 観測点の値を初期化

                    while i < point_goal:


                        if mic.volume > 50:  # マイクから突然50以上のボリュームを持つ音（器具を落とす音や悲鳴など）が入力されたときに閾値を超えたと判断し危険状況とみなす
                            emergency = True

                            if emergency:
                                subprocess.run('call.py', shell = True)

                            while emergency:
                                speaker.tune = 3000, 20


                                if button.clicked == 1:
                                    speaker.turn_off()
                                    break

                        elif 10 <= mic.volume <= 50:  # バーバルを上げる時、下げる時の呼吸の音で観測点を記録

                            if i == 0:
                                ang_vel(10, 20)
                                tilt(30)
                                vibration(5, 10)
                                ultrasonic_distance(face_distance)

                                if ang_vel_check and tilt_check and vibration_check and us_distance_check:
                                    pose_monitor_1 = True
                                    print(pose_monitor_1)

                                else:
                                    pose_monitor_1 = False
                                    print(pose_monitor_1)

                            elif i == 1:
                                ang_vel(10, 20)
                                tilt(30)
                                vibration(5, 10)

                                if ang_vel_check and tilt_check and vibration_check:
                                    pose_monitor_2 = True
                                    print(pose_monitor_2)

                                else:
                                    pose_monitor_2 = False
                                    print(pose_monitor_2)

                            i += 1
                            time.sleep(0.5)

                        else:
                            pass
                        time.sleep(0.5)

                    #############################################################　観測値サイクル終了　##############################################################
                    if pose_monitor_1 and pose_monitor_2:  # 2つの観測点を両方通った時に回数をプラス
                        pass_cnt += 1

                        if pass_cnt == reps_goal:  # 回数の目標値に達したとき、reps_limitに関係なく予めループを脱出し、1セット終了
                            display.text = "O"
                            print("o")
                            time.sleep(0.5)
                            display.clear()
                            set_failcheck[set_cnt] = True  # 該当するセットをクリアしたと判断
                            print("Congratulations! Prepare for next set.")
                            display.text = "Congratulations! Prepare for next set."
                            time.sleep(2)
                            display.clear()
                            set_cnt += 1
                            break

                        display.text = "O"
                        print("o")
                        time.sleep(0.5)
                        display.clear()
                        reps_cnt += 1

                    else:  # 2つの観測点のうち一つでも通過できなかった場合
                        display.text = "X"
                        print("x")
                        time.sleep(0.5)
                        display.clear()
                        reps_cnt += 1
                #############################reps_limit サイクル終了#######################################

                if pass_cnt < reps_goal:  # reps_limitほど繰り返したのに通過した回数が目標値を達成できなければ失敗
                    print("Failed ... Try harder next set")
                    display.text = "Failed ... Try harder next set"
                    time.sleep(2)
                    display.clear()
                    set_cnt += 1



            ############################################set_goal サイクル終了######################################



            if set_failcheck == [True]:
                print("You are a deadlift master!")
                display.text = "You are a deadlift master!"
                time.sleep(2)
                display.clear()
                break

            else:
                print("Try harder!")
                display.text = "Try harder!"
                time.sleep(2)
                display.clear()
                break
#####################################################################　デッドリフトモード終了　###############################################################




while True :
    speaker.turn_off()
    time.sleep(0.1)
    display.text = "Select mode"

    if button.clicked ==1 and 0<dial.degree<30:  # ダイアルが1番にある時ボタンを押すとベンチプレスモードに移行
        Bench_Press()

    elif button.clicked ==1 and 30<=dial.degree<60: # ダイアルが2番にある時ボタンを押すとスクワッドモードに移行
        Squat()

    elif button.clicked ==1 and 60<=dial.degree: # ダイアルが3番にある時ボタンを押すとデッドリフトモードに移行
        Deadlift()

